In [1]:
# -q: 这是pip的一个可选参数，表示"quiet"（安静模式）。它会减少命令执行时输出的信息量，使安装过程更加简洁清晰。
# -U: 这是pip的另一个可选参数，表示"upgrade"（升级）。它将确保安装的是指定包的最新版本，如果已经安装了旧版本，它将会把旧版本升级到最新版本。
# !pip install firecrawl-py faiss-cpu -q -U

In [5]:
from langchain_community.document_loaders import FireCrawlLoader


In [6]:
from langchain_huggingface import HuggingFaceEmbeddings
from tqdm import tqdm

model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings_hf = HuggingFaceEmbeddings(
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/Users/pangmengting/venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/pangmengting/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter


def build_vector_retriever(firecrawl_api_key, scrape_url):
    loader = FireCrawlLoader(api_key=firecrawl_api_key, url=scrape_url, mode="scrape")
    documents = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    docs = text_splitter.split_documents(documents)
    db = FAISS.from_documents(docs, embeddings_hf)

    return db.as_retriever()

In [4]:
import os

os.environ["FIRECRAWL_API_URL"] = 'http://localhost:3002'
firecrawl_api_key = os.getenv('FIRECRAWL_API_URL')
print(firecrawl_api_key)

http://localhost:3002


In [5]:
FIRECRAWL_API_KEY = os.getenv('FIRECRAWL_API_URL')

retriever = build_vector_retriever("FIRECRAWL_API_KEY",
                                   "https://www.youxiake.com/lines.html?id=53455&spm=eyJmcm9tIjoxMDgwLCJvcmlnaW5hbF9pZCI6MH0")


Created a chunk of size 2424, which is longer than the specified 500
Created a chunk of size 600, which is longer than the specified 500
Created a chunk of size 743, which is longer than the specified 500
Created a chunk of size 559, which is longer than the specified 500
Created a chunk of size 688, which is longer than the specified 500
Created a chunk of size 549, which is longer than the specified 500
Created a chunk of size 544, which is longer than the specified 500
Created a chunk of size 805, which is longer than the specified 500
Created a chunk of size 1246, which is longer than the specified 500
Created a chunk of size 1724, which is longer than the specified 500
Created a chunk of size 1193, which is longer than the specified 500
Created a chunk of size 513, which is longer than the specified 500


In [6]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI

# 加载当前目录下的.env文件
# load_dotenv()
# load_dotenv(override=True) 会重新读取.env
load_dotenv(override=True)

# 现在可以像访问普通环境变量一样访问.env文件中的变量了
api_key = os.getenv('OPENAI_API_KEY')
base_url = os.getenv('OPENAI_API_BASE')

print(api_key)
print(base_url)

model = ChatOpenAI(
    openai_api_base=base_url,
    openai_api_key=api_key,
    model_name="moonshot-v1-8k",
    temperature=0.7,
)

sk-UGVpjuTwo2Q8pewoqUDfckw1A0pbSDli9ElFMeS9WareKknG
https://api.moonshot.cn/v1/


In [9]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


def build_qa_chain(retriever):
    prompt = ChatPromptTemplate.from_template(
        "Using the context below to answer user's question. If you can't find information within the context, simply answer I don't know.\n\n {context} {question}")

    chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | model | StrOutputParser()

    return chain

In [10]:
chain = build_qa_chain(retriever=retriever)

In [31]:
chain.invoke("游侠客有哪些线路？")

'根据提供的网页内容，游侠客提供的主题旅行线路包括户外、摄影、亲子、深度人文、休闲度假和体育等。这些线路覆盖了不同的旅游目的地和活动类型，具体线路的详细信息没有在文档中列出。如果需要更详细的线路信息，可能需要访问游侠客的官方网站或与他们联系以获取更多信息。'

In [21]:
chain.invoke("请推荐一下特惠线路有哪些？")

'在提供的文本中，没有关于特惠线路的具体信息。因此，我无法推荐特惠线路。如果你想了解特惠线路，建议直接访问游侠客官方网站或联系客服获取最新信息。'

In [22]:
chain.invoke("腾格里五胡连穿，50KM沙漠徒步的旅游线路多少钱？")

"I don't know."

In [32]:
chain.invoke("瑜伽行·阴性之光一阶·8.6清迈阴瑜伽工作坊的具体行程是什么？")

'瑜伽行·阴性之光一阶·8.6清迈阴瑜伽工作坊的具体行程如下：\n\n天数：6天5晚\n出发：全国出发\n到达：清迈\n解散：清迈\n\nD1 第1天 各地-清迈\n- 全天：今天是自由集合的一天。组织方会根据大家当天的航班时间接机，入住清迈美好的静修中心静思之园（Suan Sati），开启6天5晚沉浸式工作坊。\n- 如果提前抵达了清迈，组织方也会在下午14点集体安排用车从清迈古城接大家到静修中心。\n- 熟悉静修中心环境，自由放松。晚上篝火聊天。\n\n重要提示：以上行程仅为参考，具体以实际签订合同为准。'

In [33]:
chain.invoke("瑜伽行·阴性之光一阶·8.6清迈阴瑜伽工作坊的第一天行程是什么，第一天住在哪里呢？")

'第一天的行程概述是自由集合的一天。根据大家的航班时间接机，入住清迈美好的静修中心静思之园（Suan Sati），开启6天5晚沉浸式工作坊。如果提前抵达了清迈，下午14点会集体安排用车从清迈古城接大家到静修中心。第一天晚上有篝火聊天的活动。所以，第一天住在清迈的静修中心静思之园（Suan Sati）。'

In [34]:
chain.invoke("据说第二天会有课程？具体有啥课程呢")

'第二天的课程安排如下：\n\n- 06:00～08:15：自我练习或参加静修中心内的瑜伽课。\n- 08:45：早餐。\n- 10:00～11:30：序列练习。\n- 11:30～12:30：理论研习，包括筋膜、阴阳、五气、疗愈原理等主题。\n- 13:00：午餐。\n- 14:30～16:30：每一个阴性体式的精讲与调整手法。\n- 16:30～17:15：教学语言与序列编排。\n- 17:17～17:30：瑜伽唱诵。\n- 18:00：晚餐。\n\n此外，课程还包括经典阴瑜伽体系的概述、身体疗愈的范围与概念、阴阳理论与呼吸法等内容。'

In [11]:
chain.invoke("据说第三天会有课程？具体有什么活动呢")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'第三天的课程安排如下：\n\n上午：\n- 06:00 - 08:15：自我练习或参加静修中心内的瑜伽课\n- 08:45：早餐\n\n中午：\n- 10:00 - 11:30：序列练习\n- 11:30 - 12:30：理论研习（筋膜、阴阳、五气、疗愈原理）\n\n下午：\n- 13:00：午餐\n- 14:30 - 16:30：每一个阴性体式的精讲与调整手法\n- 16:30 - 17:15：教学语言与序列编排\n- 17:17 - 17:30：瑜伽唱诵\n- 18:00：晚餐\n\n用餐包含三餐（需附加费用里勾选食宿），住宿是在清迈特色静修中心静思之园Suan Sati（房型根据附加费用选择）。'

In [12]:
chain.invoke("有哪些套餐？价格是多少？")

"根据提供的上下文信息，没有提及具体的套餐和价格。因此，对于问题“有哪些套餐？价格是多少？”，答案是：I don't know."

In [27]:
chain.invoke("客服电话是多少？")

"I don't know."

In [16]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter


def build_vector_retriever(firecrawl_api_key, scrape_url):
    loader = FireCrawlLoader(api_key=firecrawl_api_key, url=scrape_url, mode="crawl")
    documents = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    docs = text_splitter.split_documents(documents)
    db = FAISS.from_documents(docs, embeddings_hf)

    return db.as_retriever()


In [17]:

retriever2 = build_vector_retriever("FIRECRAWL_API_KEY",
                                    "https://www.youxiake.com")

chain2 = build_qa_chain(retriever=retriever2)

Created a chunk of size 2090, which is longer than the specified 1000
Created a chunk of size 1905, which is longer than the specified 1000
Created a chunk of size 2052, which is longer than the specified 1000
Created a chunk of size 2424, which is longer than the specified 1000
Created a chunk of size 1495, which is longer than the specified 1000
Created a chunk of size 2424, which is longer than the specified 1000
Created a chunk of size 3949, which is longer than the specified 1000
Created a chunk of size 1709, which is longer than the specified 1000
Created a chunk of size 1961, which is longer than the specified 1000
Created a chunk of size 1944, which is longer than the specified 1000
Created a chunk of size 1948, which is longer than the specified 1000
Created a chunk of size 2015, which is longer than the specified 1000
Created a chunk of size 1946, which is longer than the specified 1000
Created a chunk of size 1971, which is longer than the specified 1000
Created a chunk of s

In [18]:
chain2.invoke("游侠客最便宜的是哪条线路？")

"I don't know."

In [26]:
chain2.invoke("客服电话是多少？")

'根据上述提供的文档内容，客服电话是400-670-6300。'

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter


def build_vector_retriever(firecrawl_api_key, scrape_url):
    loader = FireCrawlLoader(api_key=firecrawl_api_key, url=scrape_url, mode="crawl")
    documents = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    docs = text_splitter.split_documents(documents)
    db = FAISS.from_documents(docs, embeddings_hf)

    return db.as_retriever()

In [2]:
def loader_data(firecrawl_api_key, scrape_url):
    loader = FireCrawlLoader(api_key=firecrawl_api_key, url=scrape_url, mode="crawl")
    documents = loader.load()
    return documents

In [3]:
def split(documents):
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    docs = text_splitter.split_documents(documents)
    return docs

In [7]:
documents = loader_data("FIRECRAWL_API_KEY", "https://bbs.youxiake.com/mdd/9558.html")
docs = split(documents)
texts = [doc.page_content for doc in docs]
metadatas = [doc.metadata for doc in docs]

Exception: Unexpected error occurred while trying to start crawl job. Status code: 401

In [ ]:
import os
import chromadb
from chromadb import Settings

from pathlib import Path

# Path(__file__).parent：这一行创建了一个 Path 对象，表示当前脚本文件所在的目录。
# __file__ 是一个内置变量，指向当前脚本的路径。.parent 方法返回该路径的父目录。
# BACKEND_DIR = Path(__file__).parent
# DATA_DIR = Path(os.getenv("DATA_DIR", BACKEND_DIR / "data")).resolve()

path = '/Users/pangmengting/Documents/workspace/python-learning/langchain/chromadb'
CHROMA_DATA_PATH = f"{path}/vector_db"
CHROMA_TENANT = os.environ.get("CHROMA_TENANT", chromadb.DEFAULT_TENANT)
CHROMA_DATABASE = os.environ.get("CHROMA_DATABASE", chromadb.DEFAULT_DATABASE)

# 初始化 chromadb
CHROMA_CLIENT = chromadb.PersistentClient(
    path=CHROMA_DATA_PATH,
    settings=Settings(allow_reset=True, anonymized_telemetry=False),
    tenant=CHROMA_TENANT,
    database=CHROMA_DATABASE,
)